# Gaussian process with Spike and Slab ARD prior - Stage 1 Savitsky experiment
## Zero-temperature-MF VI algorithm (continuous spike) - non additive kernels

### (1) Importing and defining all required functions

In [1]:
"""
Importing required libraries and defining key functions
"""
# Requirements for algorithms
import numpy as np
import scipy.special as sp
from scipy.spatial.distance import cdist 

# Requirements For plots and diagnostics
import matplotlib.pyplot as plt
import scipy.stats as sps
import time
from sklearn.metrics import roc_auc_score
from datetime import date
import inspect
import pickle
from IPython.display import clear_output

# Algorithm functions
import os
os.chdir('C:/Users/hughw/Documents/MSC project/GP algorithms/Master function files')
from GP_funcs_ZTMFSS import kernel_funcs
from GP_funcs_ZTMFSS import model_funcs
from GP_funcs_ZTMFSS import draw_GP
from GP_funcs_ZTMFSS import fit
from GP_funcs_ZTMFSS import diagnostics
from GP_funcs_ZTMFSS import simulations
from functools import partial
os.chdir('C:/Users/hughw/Documents/MSC project/Simulation results')

### (2) Setting simulation parameters and models

In [2]:
"""
Simulation controls
"""
# Simulation settings
n=100
ntest=20
p=1000
q=6
correlation = False
nruns = 100

# Model run settings
l_init = 0.01
beta2=0.99
m = 2
nmodels = 6
VS_threshs = [[0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99],
             [0.5,0.9,0.95,0.99]]
iter_remove = [False,False,False,False,False,False]
sampling_strat = ["unif", "unif", "unif", "unif", "unif", "unif"]
minibatch_size = [50, 50,25,25,n,n]
GPtol=[1e-5,1e-5,1e-5,1e-5,1e-5,1e-5]
SS_GP=[True,True,True,True,True,True]
MC_pred = [False,False,False,False,False,False]
predict_selected = [False,False,False,False,False,False]
post_fit = [False,False,False,False,False,False]
train = [True,False,True, False, True,False] 
step = [0.05,0.05,0.05,0.05,0.05,0.05]
hyper_opt = [True,True,True,True,True,True]
model_select = [False,True,False,True,False,True]
post_var = [True,True,True,True,True,True]
model_weights = ["elpd", "elpd","elpd", "elpd","elpd", "elpd"]
min_VBEM_iter = [3,3,3,3,3,3]
max_VBEM_iter = [10,10,10,10,10,10]
gp_iter = [100,100,100,100,100,100]
opt = ["amsgrad", "amsgrad", "amsgrad", "amsgrad", "amsgrad", "amsgrad"]

t = len(VS_threshs[0])
kern=kernel_funcs.gaussian
grad_kern=kernel_funcs.grad_gaussian
NN_pred = False
newsumgrads = False
VBtol=0.1/p
temp=1
# SEE BELOW FOR ARGVALS

# Storage objects
Runtime=np.zeros((nruns, m))
Lambda = np.zeros((nruns, m, p))
L = np.zeros((nruns, m, p))
L1norm=np.zeros((nruns, m))
L2norm=np.zeros((nruns, m))
MSE_F=np.zeros((nruns, m))
MSE_Y=np.zeros((nruns,m))
Acc=np.zeros((nruns,m,t))
Weighted_Acc=np.zeros((nruns,m,t))
TPR=np.zeros((nruns,m,t))
TNR=np.zeros((nruns,m,t))
PPV=np.zeros((nruns,m,t))
NPV=np.zeros((nruns,m,t))
MCC=np.zeros((nruns,m,t))
AUC=np.zeros((nruns,m))

### (3) Running algorithm iterations, saving and displaying results

In [3]:
np.random.seed(8750)
runlist = np.random.choice(1000,100,False) # Choose 100 random trials
for run in range(len(runlist)):
    
    """
    Generating data and scaling data
    """
    lselect=[]
    np.random.seed(runlist[run]) # Fixing trial seed
    t=time.time()
    Y,F,X,e,sigma,select=draw_GP.draw_parametric_savitsky(n,ntest,p,q, correlation)
    
    Y = Y.reshape(n+ntest,1)
    F = F.reshape(n+ntest,1)
    
    #Y = (Y-Y.mean())/Y.var()**0.5
    #F = (F-F.mean())/F.var()**0.5
    X = (X-X.mean(0))/X.var(0)**0.5

    # Getting training and test set
    ytest=Y[n:]
    Xtest=X[n:]
    ftest=F[n:]
    y=Y[:n]
    X=X[:n]
    f=F[:n]
    print("data generated")
    if lselect:
        print("Length-scales are: ",lselect[select])
    print("Noise variance is: ",sigma**2)
    print("Average data variance is: ", np.mean(np.var(X,0)))
    print(time.time()-t)
    
    """
    Running algorithm
    """
    args=[]
    arg_vals =[]
    for i in range(nmodels):
        args.append(["seed", "subsample", "Beta2", "ELBO_sample", "learn_rate", "ltrue", "learn_spike", "min_VBEM_iter", "init_GP_iter", "max_VBEM_iter", "GP_fit_tol", "VBEM_tol", "print_VBEM",
             "s0", "sig0", "newsumgrads","temp","v0","v1", "max_GP_fit_iter", "iter_remove", "learn_rate_mult", "sampling_strat", "final_prune"])
        arg_vals.append([1, minibatch_size[i], beta2,  min(1000,n), step[i], [], False, min_VBEM_iter[i],gp_iter[i] , max_VBEM_iter[i], GPtol[i], VBtol, False, 
                 np.var(y), np.var(y)**0.5, newsumgrads,temp,1e+4,1e-4, gp_iter[i], iter_remove[i], 1, "unif",True])

    testing_algorithm = partial(diagnostics.get_pred_posterior_GP,reg=0.01,kern = kernel_funcs.gaussian)

    Runtime[run], Lambda[run], L[run], L1norm[run], L2norm[run], MSE_F[run], MSE_Y[run], Acc[run], Weighted_Acc[run], TPR[run], TNR[run], PPV[run], NPV[run], AUC[run], MCC[run] = simulations.do_simulation_VBEMSSGP(
                               y, X, ftest, ytest, Xtest, q, algorithm_training = fit.VB_EM_GP_SS, algorithm_testing = testing_algorithm, post_var = post_var,
                               nmodels = m, args = args, arg_vals = arg_vals, SS_GP = SS_GP, hyper_opt = hyper_opt, train = train,
                                hyper_arg = ["v0","v1"], hyper_vals = [1e+4*2**np.linspace(np.log2(100),-np.log2(100),11),2**np.linspace(np.log2(100),-np.log2(100),11)], order_relevant_vars = False, order_irrelevant_vars = False, 
                                VS_threshs = VS_threshs, select = select, predict_selected = predict_selected, ltrue=lselect, MC_pred = MC_pred, post_fit = post_fit, model_select = model_select,
                                model_weighting = model_weights)
    
    print("RUN {0}".format(run))
    print("Runtime mean is:", Runtime[:run+1].mean(0))
    print("Weighted accuracy mean is:", Weighted_Acc[:run].mean(0))
    print("TPR mean is:", TPR[:run+1].mean(0))
    print("PPV mean is:", PPV[:run+1].mean(0))
    print("MCC mean is:", MCC[:run+1].mean(0))
    print("L1norm mean is:", L1norm[:run+1].mean(0))
    print("L2norm mean is:", L2norm[:run+1].mean(0))
    print("MSE_F mean is:", MSE_F[:run+1].mean(0))
    print("MSE_Y mean is:", MSE_Y[:run+1].mean(0), "\n")
    
    print("Runtime is:", Runtime[run])
    print("Weighted accuracy is:", Weighted_Acc[run])
    print("TPR is:", TPR[run])
    print("PPV is:", PPV[run])
    print("MCC is:", MCC[run])
    print("L1norm is:", L1norm[run])
    print("L2norm is:", L2norm[run])
    print("MSE_F is:", MSE_F[run])
    print("MSE_Y is:", MSE_Y[run], "\n")
    

data generated
Noise variance is:  0.0025000000000000005
Average data variance is:  0.9969892827714567
0.002992391586303711
run time is : 27.051925897598267
run time is : 21.577027082443237
run time is : 18.408673763275146
run time is : 15.856580257415771
run time is : 12.274930238723755
run time is : 30.20845627784729
run time is : 31.382245302200317
run time is : 32.16075086593628
run time is : 30.265522956848145
run time is : 32.2826247215271
run time is : 29.24168038368225


C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T
<ipython-input-3-dbf07814af72>:56: RuntimeWarning: Mean of empty slice.
  print("Weighted accuracy mean is:", Weighted_Acc[:run].mean(0))
C:\Users\hughw\anaconda3\lib\site-packages\numpy\core\_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


RUN 0
Runtime mean is: [281.22153568 281.25041914]
Weighted accuracy mean is: [[nan nan nan nan]
 [nan nan nan nan]]
TPR mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
PPV mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
MCC mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
L1norm mean is: [0. 0.]
L2norm mean is: [0. 0.]
MSE_F mean is: [0.00268586 0.00293515]
MSE_Y mean is: [0.00309066 0.00334011] 

Runtime is: [281.22153568 281.25041914]
Weighted accuracy is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
TPR is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
PPV is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
MCC is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
L1norm is: [0. 0.]
L2norm is: [0. 0.]
MSE_F is: [0.00268586 0.00293515]
MSE_Y is: [0.00309066 0.00334011] 

data generated
Noise variance is:  0.0025000000000000005
Average data variance is:  0.9980154643063102
0.002991914749145508
run time is : 33.47464036941528
run time is : 24.71651577949524
run time is : 16.066400289535522
run time is : 15.579273223876953
run time is : 15.910861253738403
run time is : 30.5

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


RUN 1
Runtime mean is: [290.32473958 290.35167837]
Weighted accuracy mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
TPR mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
PPV mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
MCC mean is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
L1norm mean is: [0. 0.]
L2norm mean is: [0. 0.]
MSE_F mean is: [0.00196347 0.00213709]
MSE_Y mean is: [0.00248181 0.00254848] 

Runtime is: [299.42794347 299.4529376 ]
Weighted accuracy is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
TPR is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
PPV is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
MCC is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
L1norm is: [0. 0.]
L2norm is: [0. 0.]
MSE_F is: [0.00124108 0.00133903]
MSE_Y is: [0.00187296 0.00175685] 

data generated
Noise variance is:  0.0025000000000000005
Average data variance is:  1.0006914241839122
0.0029914379119873047
run time is : 30.8179612159729
run time is : 21.366543769836426
run time is : 21.507104635238647
run time is : 21.582444190979004
run time is : 15.584325075149536
run time is : 31.94524455

...

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


RUN 97
Runtime mean is: [282.68998173 282.70216968]
Weighted accuracy mean is: [[0.99728785 0.98366142 0.97592946 0.94587629]
 [0.99200357 0.99115483 0.99115483 0.99115483]]
TPR mean is: [[0.99489796 0.96768707 0.95238095 0.89285714]
 [0.98469388 0.9829932  0.9829932  0.9829932 ]]
PPV mean is: [[0.98303057 0.99538387 0.99538387 1.        ]
 [0.97231425 0.97388241 0.97388241 0.97388241]]
MCC mean is: [[0.98747818 0.97981247 0.97126369 0.94009784]
 [0.97437951 0.97412891 0.97412891 0.97412891]]
L1norm mean is: [0. 0.]
L2norm mean is: [0. 0.]
MSE_F mean is: [0.01075294 0.0108829 ]
MSE_Y mean is: [0.01328306 0.01349513] 

Runtime is: [258.17116499 258.17044687]
Weighted accuracy is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
TPR is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
PPV is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
MCC is: [[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
L1norm is: [0. 0.]
L2norm is: [0. 0.]
MSE_F is: [0.00702434 0.00604605]
MSE_Y is: [0.01409331 0.01303953] 

data generated
Noise variance is:  0.0025000000000000005

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


RUN 98
Runtime mean is: [282.94503225 282.9571867 ]
Weighted accuracy mean is: [[0.99731553 0.98382814 0.97617508 0.94642857]
 [0.99208516 0.99124509 0.99124509 0.99124509]]
TPR mean is: [[0.99326599 0.96632997 0.95117845 0.89225589]
 [0.98316498 0.98148148 0.98148148 0.98148148]]
PPV mean is: [[0.97670848 0.9954305  0.9954305  1.        ]
 [0.96529873 0.96685105 0.96685105 0.96685105]]
MCC mean is: [[0.9829764  0.97913166 0.97066923 0.93981819]
 [0.96935001 0.96910194 0.96910194 0.96910194]]
L1norm mean is: [0. 0.]
L2norm mean is: [0. 0.]
MSE_F mean is: [0.01210395 0.01286993]
MSE_Y mean is: [0.01458724 0.01541773] 

Runtime is: [307.93998337 307.94885492]
Weighted accuracy is: [[0.9121395  0.91666667 0.91666667 0.91666667]
 [0.91012743 0.91012743 0.91012743 0.91012743]]
TPR is: [[0.83333333 0.83333333 0.83333333 0.83333333]
 [0.83333333 0.83333333 0.83333333 0.83333333]]
PPV is: [[0.35714286 1.         1.         1.        ]
 [0.27777778 0.27777778 0.27777778 0.27777778]]
MCC is: [[0

C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:55: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,X, metric = "seuclidean", V = 1/l**2),s)
C:\Users\hughw\Documents\MSC project\GP algorithms\Master function files\GP_funcs_ZTMFSS.py:62: RuntimeWarning: divide by zero encountered in true_divide
  return kern(cdist(X,Xtest, metric = "seuclidean", V = 1/l**2),s).T


In [6]:
namelist = ["Runtime", "MSE_F", "MSE_Y", "Acc", "Weighted_Acc", "TPR", "TNR", "PPV", "NPV", "AUC", "MCC"]
objlist = [Runtime, MSE_F, MSE_Y, Acc, Weighted_Acc, TPR, TNR, PPV, NPV, AUC, MCC]
#iters = np.random.choice(1000,100,False)
iters = np.linspace(0,99,100).astype(int)

for i in range(len(objlist)):
    print("Mean {0} is:".format(namelist[i]), np.mean(objlist[i][iters],0))

print("\n")
for i in range(len(objlist)):
    print("Median {0} is:".format(namelist[i]), np.median(objlist[i][iters],0))

print("\n")
quant = 0.25
for i in range(len(objlist)):
    if namelist[i] in ["Runtime", "MSE_F", "MSE_Y"]:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],1-quant,0))
    else:
        print("{1} quantile {0} is:".format(namelist[i], quant), np.quantile(objlist[i][iters],quant,0))

Mean Runtime is: [282.68503147 282.69698616]
Mean MSE_F is: [0.01201429 0.01277142]
Mean MSE_Y is: [0.01453032 0.01535132]
Mean Acc is: [[0.99961 0.99977 0.99968 0.99936]
 [0.99926 0.99927 0.99927 0.99927]]
Mean Weighted_Acc is: [[0.99649061 0.98331824 0.97581824 0.94666667]
 [0.99134474 0.99052146 0.99052146 0.99052146]]
Mean TPR is: [[0.99333333 0.96666667 0.95166667 0.89333333]
 [0.98333333 0.98166667 0.98166667 0.98166667]]
Mean TNR is: [[0.99964789 0.99996982 0.99996982 1.        ]
 [0.99935614 0.99937626 0.99937626 0.99937626]]
Mean PPV is: [[0.97694139 0.99547619 0.99547619 1.        ]
 [0.96564574 0.96718254 0.96718254 0.96718254]]
Mean NPV is: [[0.99995944 0.99979913 0.99970877 0.99935743]
 [0.99989901 0.99988901 0.99988901 0.99988901]]
Mean AUC is: [0.99598424 0.99634222]
Mean MCC is: [[0.98314664 0.97934034 0.97096253 0.94042001]
 [0.96965651 0.96941092 0.96941092 0.96941092]]


Median Runtime is: [281.30807292 281.32798517]
Median MSE_F is: [0.00356603 0.00306474]
Median MS

In [7]:
Output = {"Runtime" : Runtime, "Lambda" : Lambda, "L" : L, "L1norm" : L1norm, "L2norm" : L2norm, "MSE_F" : MSE_F
        , "MSE_Y" : MSE_Y, "Acc" : Acc, "Weighted_Acc" : Weighted_Acc, "TPR" :TPR, "TNR" : TNR, "PPV" : PPV, "NPV" : NPV, "AUC" : AUC, "MCC" : MCC}
String = "Stage1_Savitsky_ZTnoremove_{0}_l0={1}_b2={2}_newgrads={3}_predselect={4}_MCpred={5}_hyperopt={11}_minibatch={12}_n={6}_p={7}_q={8}_kern={9}_runs={10}".format(
        date.today(), l_init, beta2, newsumgrads, predict_selected[1],MC_pred[1],n,p,q,str(kern)[23:28], nruns, hyper_opt[1],minibatch_size[1])
np.save(String, Output) # saving